<a href="https://colab.research.google.com/github/SushilKokare/AtiNeu_Project/blob/main/ensemble_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,Flatten, Dropout, Input, Average, GlobalAveragePooling2D
from keras.models import Model

In [4]:

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications.vgg16 import VGG16

In [5]:
input_shape = (224,224,3)
num_classes = 2
batch_size = 32

In [6]:
train_dir = '/content/drive/MyDrive/AtiNeu Project/Data/MRI/Brain_Data_Organised/Train'
test_dir = '/content/drive/MyDrive/AtiNeu Project/Data/MRI/Brain_Data_Organised/Test'

In [7]:
train_datagen = ImageDataGenerator (rescale = 1./225,)
test_datagen = ImageDataGenerator (rescale = 1./225)

In [8]:
train_gen  = train_datagen.flow_from_directory(
    train_dir,
    target_size = input_shape[:2],
    batch_size = batch_size,
    class_mode = 'categorical'
)

Found 2251 images belonging to 2 classes.


In [9]:
test_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size = input_shape[:2],
    batch_size = batch_size,
    class_mode = 'categorical',
    color_mode='rgb'
)

Found 250 images belonging to 2 classes.


In [10]:
train_gen.image_shape

(224, 224, 3)

In [11]:
test_gen.image_shape


(224, 224, 3)

In [12]:

_resnet50 = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
_densenet121 = DenseNet121(weights='imagenet', include_top=False, input_shape=input_shape)
_vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)


58889256/58889256 [==============================] - 4s 0us/step


In [13]:
_resnet50_output = GlobalAveragePooling2D()(_resnet50.output)
_resnet50_output = Dense(1,activation='sigmoid')(_resnet50_output)

In [14]:
_resnet50_output

<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'dense')>

In [15]:
_densenet121_output = GlobalAveragePooling2D()(_densenet121.output)
_densenet121_output = Dense(1,activation='sigmoid')(_densenet121_output)

In [16]:
_densenet121_output

<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'dense_1')>

In [17]:
_vgg16_output = GlobalAveragePooling2D()(_vgg16.output)
_vgg16_output = Dense(1,activation='sigmoid')(_vgg16_output)

In [18]:
_vgg16_output

<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'dense_2')>

In [19]:
inputs = Input(shape=(224,224,3))

_resnet50_output = _resnet50(inputs)
_densenet121_output = _densenet121(inputs)
_vgg16_output = _vgg16(inputs)


In [20]:
_resnet50_output

<KerasTensor: shape=(None, 7, 7, 2048) dtype=float32 (created by layer 'resnet50')>

In [21]:
_densenet121_output

<KerasTensor: shape=(None, 7, 7, 1024) dtype=float32 (created by layer 'densenet121')>

In [22]:
_vgg16_output

<KerasTensor: shape=(None, 7, 7, 512) dtype=float32 (created by layer 'vgg16')>

In [23]:
avg1 = Average()([_resnet50_output])

In [24]:
avg2 = Average()([_densenet121_output])

In [25]:
avg3 = Average()([_vgg16_output])

In [26]:
output = tf.keras.layers.concatenate([avg1,avg2,avg3])

In [27]:
# model = Model(inputs=[_resnet50_output,_densenet121_output,_vgg16_output],outputs=output)

In [28]:
model = Model(inputs=inputs,outputs=output)

In [29]:

# output = Average()([_resnet50_output,_densenet121_output,_vgg16_output])
 
# model = Model(inputs=inputs,outputs=output)

In [30]:

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [31]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 resnet50 (Functional)          (None, 7, 7, 2048)   23587712    ['input_4[0][0]']                
                                                                                                  
 densenet121 (Functional)       (None, 7, 7, 1024)   7037504     ['input_4[0][0]']                
                                                                                                  
 vgg16 (Functional)             (None, 7, 7, 512)    14714688    ['input_4[0][0]']            

In [32]:
# history = model.fit(train_gen,
#           epochs=10,)
